In [1]:
import time

from dgl.dataloading import GraphDataLoader
import torch

from prob import VRPDGLDataset
from solver import return2Depot
from utils import checkValid, routesPlot

Using backend: pytorch


In [2]:
def run(size):
    """
    A function to evaluate different algorithms

    Args:
        size(int): graph size
        algo (str): name of algorithm
        solver_args (tuple): args of solver
    """
    cnt = 0
    # device
    print("Device:")
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    print("  {}".format(device))
    # load test data
    print("\nLoad data...")
    print("  Graph size: {}".format(size))
    data = VRPDGLDataset(size=size, num_samples=1000)
    batch = 1
    dataloader = GraphDataLoader(data, batch_size=batch, shuffle=False)
    # init solver
    solver = return2Depot(size)
    # solve
    print("\nEvaluating:")
    for batch_data, batch_graph in dataloader:
        cnt += 1
        # to device
        batch_graph = batch_graph.to(device)
        batch_data["loc"] = batch_data["loc"].to(device)
        batch_data["demand"] = batch_data["demand"].to(device)
        batch_data["depot"] = batch_data["depot"].to(device)
        tick = time.time()
        batch_routes, batch_objs = solver.solve(batch_data, batch_graph)
        tock = time.time()
        for i in range(batch):
            obj = batch_objs[i]
            routes = batch_routes[i]
            depot = batch_data["depot"].cpu().detach().numpy()[i]
            loc = batch_data["loc"].cpu().detach().numpy()[i]
            demand = batch_data["demand"].cpu().detach().numpy()[i]
            # print results
            print("Obj Val: {:.4f}".format(obj))
            print("Routes:")
            for r in routes:
                print("  ", "->".join(map(str, r)))
            print("Time elpased: {:.4f} sec".format((tock - tick) / batch))
            # check valid
            assert checkValid(routes, depot, loc, demand), "Infeasible solution."
            # vis
            routesPlot(routes, depot, loc)
        # stop
        if cnt >= 3:
            break
        print()

In [5]:
run(size=50)

  1%|          | 6/1000 [00:00<00:18, 54.54it/s]

Device:
  cpu

Load data...
  Graph size: 50


100%|██████████| 1000/1000 [00:17<00:00, 55.81it/s]



Loading routing agent...
  [*] Loading model from ./attention_model/pretrained_tsp/tsp_50/epoch-99.pt

Loading return2depot agent...


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.